# Image processing with satellite data

## Large-scale computations on a Kubernetes cluster (GKE on GCP)

This notebook performs calculations with a GeoTIFF dataset using XArray and Dask. We load and rescale Landsat 8 images and compute the normalized difference vegetation index (NDVI), which distinguishes green vegetation from areas of bare land or water.

We'll use 200 images of the Denver, USA area taken from March 2017 to November 2021.

![RGB image](https://landsat-pds.s3.amazonaws.com/c1/L8/033/033/LC08_L1TP_033033_20180706_20180717_01_T1/LC08_L1TP_033033_20180706_20180717_01_T1_thumb_small.jpg)

## Step 1: Import packages

In [ ]:
import dask
import json
import os
import rasterio
import requests
import rioxarray

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

## Step 2: Define input data/images

We are using 200 images from the [Landsat dataset on GCP](https://cloud.google.com/storage/docs/public-datasets/landsat) and each band is available as a separate GeoTIFF file.

In [ ]:
images = [
"LC08_L1TP_033033_20130318_20170310_01_T1", "LC08_L1TP_033033_20130419_20170310_01_T1", "LC08_L1TP_033033_20130505_20170310_01_T1",
"LC08_L1TP_033033_20130521_20170310_01_T1", "LC08_L1TP_033033_20130606_20170310_01_T1", "LC08_L1TP_033033_20130622_20170309_01_T1",
"LC08_L1TP_033033_20130708_20170309_01_T1", "LC08_L1TP_033033_20130724_20170309_01_T1", "LC08_L1TP_033033_20130809_20170309_01_T1",
"LC08_L1TP_033033_20130825_20170309_01_T1", "LC08_L1TP_033033_20130910_20170309_01_T1", "LC08_L1TP_033033_20130926_20170308_01_T1",
"LC08_L1TP_033033_20131012_20170308_01_T1", "LC08_L1TP_033033_20131028_20170308_01_T1", "LC08_L1TP_033033_20131113_20170307_01_T1",
"LC08_L1TP_033033_20131129_20170307_01_T1", "LC08_L1TP_033033_20131215_20170307_01_T1", "LC08_L1TP_033033_20131231_20170307_01_T1",
"LC08_L1TP_033033_20140116_20170308_01_T1", "LC08_L1TP_033033_20140201_20170307_01_T1", "LC08_L1TP_033033_20140217_20170307_01_T1",
"LC08_L1TP_033033_20140305_20170307_01_T1", "LC08_L1TP_033033_20140321_20170307_01_T1", "LC08_L1TP_033033_20140406_20170307_01_T1",
"LC08_L1TP_033033_20140422_20170306_01_T1", "LC08_L1TP_033033_20140508_20170307_01_T1", "LC08_L1TP_033033_20140524_20170307_01_T1",
"LC08_L1TP_033033_20140609_20170305_01_T1", "LC08_L1TP_033033_20140625_20170304_01_T1", "LC08_L1TP_033033_20140711_20170304_01_T1",
"LC08_L1TP_033033_20140727_20170304_01_T1", "LC08_L1TP_033033_20140812_20170304_01_T1", "LC08_L1TP_033033_20140828_20170303_01_T1",
"LC08_L1TP_033033_20140913_20170303_01_T1", "LC08_L1TP_033033_20140929_20170303_01_T1", "LC08_L1TP_033033_20141015_20170303_01_T1",
"LC08_L1TP_033033_20141031_20170303_01_T1", "LC08_L1TP_033033_20141116_20170302_01_T1", "LC08_L1TP_033033_20141202_20170302_01_T1",
"LC08_L1TP_033033_20141218_20170302_01_T1", "LC08_L1TP_033033_20150103_20170302_01_T1", "LC08_L1TP_033033_20150119_20170302_01_T1",
"LC08_L1TP_033033_20150119_20180131_01_T1", "LC08_L1TP_033033_20150204_20170301_01_T1", "LC08_L1TP_033033_20150220_20170228_01_T1",
"LC08_L1TP_033033_20150308_20170301_01_T1", "LC08_L1TP_033033_20150324_20170301_01_T1", "LC08_L1TP_033033_20150409_20170301_01_T1",
"LC08_L1TP_033033_20150409_20180131_01_T1", "LC08_L1TP_033033_20150425_20170301_01_T1", "LC08_L1TP_033033_20150511_20170301_01_T1",
"LC08_L1TP_033033_20150527_20170301_01_T1", "LC08_L1TP_033033_20150612_20170301_01_T1", "LC08_L1TP_033033_20150628_20170301_01_T1",
"LC08_L1TP_033033_20150714_20170226_01_T1", "LC08_L1TP_033033_20150730_20170226_01_T1", "LC08_L1TP_033033_20150815_20170226_01_T1",
"LC08_L1TP_033033_20150831_20170301_01_T1", "LC08_L1TP_033033_20150831_20180131_01_T1", "LC08_L1TP_033033_20150916_20170302_01_T1",
"LC08_L1TP_033033_20151002_20170225_01_T1", "LC08_L1TP_033033_20151018_20170225_01_T1", "LC08_L1TP_033033_20151103_20170225_01_T1",
"LC08_L1TP_033033_20151119_20170225_01_T1", "LC08_L1TP_033033_20151205_20170224_01_T1", "LC08_L1TP_033033_20151221_20170224_01_T1",
"LC08_L1TP_033033_20160106_20170224_01_T1", "LC08_L1TP_033033_20160122_20170224_01_T1", "LC08_L1TP_033033_20160207_20170224_01_T1",
"LC08_L1TP_033033_20160223_20170224_01_T1", "LC08_L1TP_033033_20160310_20170224_01_T1", "LC08_L1TP_033033_20160411_20170223_01_T1",
"LC08_L1TP_033033_20160427_20170223_01_T1", "LC08_L1TP_033033_20160513_20170223_01_T1", "LC08_L1TP_033033_20160529_20170223_01_T1",
"LC08_L1TP_033033_20160614_20170220_01_T1", "LC08_L1TP_033033_20160630_20170221_01_T1", "LC08_L1TP_033033_20160716_20170223_01_T1",
"LC08_L1TP_033033_20160801_20170222_01_T1", "LC08_L1TP_033033_20160817_20170221_01_T1", "LC08_L1TP_033033_20160902_20170221_01_T1",
"LC08_L1TP_033033_20160918_20170221_01_T1", "LC08_L1TP_033033_20161004_20170220_01_T1", "LC08_L1TP_033033_20161020_20170219_01_T1",
"LC08_L1TP_033033_20161105_20170219_01_T1", "LC08_L1TP_033033_20161121_20170219_01_T1", "LC08_L1TP_033033_20161207_20170219_01_T1",
"LC08_L1TP_033033_20161223_20170218_01_T1", "LC08_L1TP_033033_20170108_20170218_01_T1", "LC08_L1TP_033033_20170124_20170218_01_T1",
"LC08_L1TP_033033_20170209_20170217_01_T1", "LC08_L1TP_033033_20170225_20170316_01_T1", "LC08_L1TP_033033_20170313_20170328_01_T1",
"LC08_L1TP_033033_20170329_20170414_01_T1", "LC08_L1TP_033033_20170414_20170501_01_T1", "LC08_L1TP_033033_20170430_20170515_01_T1",
"LC08_L1TP_033033_20170516_20170525_01_T1", "LC08_L1TP_033033_20170516_20180125_01_T1", "LC08_L1TP_033033_20170601_20170615_01_T1",
"LC08_L1TP_033033_20170617_20170629_01_T1", "LC08_L1TP_033033_20170703_20170715_01_T1", "LC08_L1TP_033033_20170719_20170728_01_T1",
"LC08_L1TP_033033_20170804_20170812_01_T1", "LC08_L1TP_033033_20170820_20170826_01_T1", "LC08_L1TP_033033_20170905_20170917_01_T1",
"LC08_L1TP_033033_20170921_20171012_01_T1", "LC08_L1TP_033033_20171007_20171023_01_T1", "LC08_L1TP_033033_20171023_20171107_01_T1",
"LC08_L1TP_033033_20171108_20171121_01_T1", "LC08_L1TP_033033_20171124_20171206_01_T1", "LC08_L1TP_033033_20171210_20171223_01_T1",
"LC08_L1TP_033033_20171226_20180103_01_T1", "LC08_L1TP_033033_20180111_20180119_01_T1", "LC08_L1TP_033033_20180127_20180207_01_T1",
"LC08_L1TP_033033_20180212_20180222_01_T1", "LC08_L1TP_033033_20180228_20180308_01_T1", "LC08_L1TP_033033_20180316_20180402_01_T1",
"LC08_L1TP_033033_20180401_20180416_01_T1", "LC08_L1TP_033033_20180417_20180501_01_T1", "LC08_L1TP_033033_20180503_20180516_01_T1",
"LC08_L1TP_033033_20180519_20180605_01_T1", "LC08_L1TP_033033_20180604_20180615_01_T1", "LC08_L1TP_033033_20180620_20180703_01_T1",
"LC08_L1TP_033033_20180706_20180717_01_T1", "LC08_L1TP_033033_20180722_20180731_01_T1", "LC08_L1TP_033033_20180807_20180815_01_T1",
"LC08_L1TP_033033_20180823_20180829_01_T1", "LC08_L1TP_033033_20180908_20180912_01_T1", "LC08_L1TP_033033_20180924_20180929_01_T1",
"LC08_L1TP_033033_20181026_20181115_01_T1", "LC08_L1TP_033033_20181127_20181211_01_T1", "LC08_L1TP_033033_20181213_20181227_01_T1",
"LC08_L1TP_033033_20181229_20190130_01_T1", "LC08_L1TP_033033_20190114_20190131_01_T1", "LC08_L1TP_033033_20190130_20190206_01_T1",
"LC08_L1TP_033033_20190215_20190222_01_T1", "LC08_L1TP_033033_20190303_20190309_01_T1", "LC08_L1TP_033033_20190319_20190325_01_T1",
"LC08_L1TP_033033_20190404_20190422_01_T1", "LC08_L1TP_033033_20190420_20190507_01_T1", "LC08_L1TP_033033_20190506_20190521_01_T1",
"LC08_L1TP_033033_20190522_20190604_01_T1", "LC08_L1TP_033033_20190607_20190619_01_T1", "LC08_L1TP_033033_20190623_20190704_01_T1",
"LC08_L1TP_033033_20190709_20190719_01_T1", "LC08_L1TP_033033_20190725_20190801_01_T1", "LC08_L1TP_033033_20190810_20190820_01_T1",
"LC08_L1TP_033033_20190826_20190903_01_T1", "LC08_L1TP_033033_20190911_20190917_01_T1", "LC08_L1TP_033033_20190927_20191017_01_T1",
"LC08_L1TP_033033_20191013_20191018_01_T1", "LC08_L1TP_033033_20191114_20191202_01_T1", "LC08_L1TP_033033_20191130_20191216_01_T1",
"LC08_L1TP_033033_20191216_20191226_01_T1", "LC08_L1TP_033033_20191216_20201022_01_T1", "LC08_L1TP_033033_20200101_20200113_01_T1",
"LC08_L1TP_033033_20200117_20200128_01_T1", "LC08_L1TP_033033_20200202_20200211_01_T1", "LC08_L1TP_033033_20200218_20200225_01_T1",
"LC08_L1TP_033033_20200305_20200314_01_T1", "LC08_L1TP_033033_20200321_20200326_01_T1", "LC08_L1TP_033033_20200406_20200410_01_T1",
"LC08_L1TP_033033_20200422_20200508_01_T1", "LC08_L1TP_033033_20200508_20200526_01_T1", "LC08_L1TP_033033_20200524_20200607_01_T1",
"LC08_L1TP_033033_20200609_20200626_01_T1", "LC08_L1TP_033033_20200625_20200707_01_T1", "LC08_L1TP_033033_20200711_20200722_01_T1",
"LC08_L1TP_033033_20200727_20200807_01_T1", "LC08_L1TP_033033_20200812_20200822_01_T1", "LC08_L1TP_033033_20200828_20200905_01_T1",
"LC08_L1TP_033033_20200913_20200920_01_T1", "LC08_L1TP_033033_20200929_20201007_01_T1", "LC08_L1TP_033033_20201015_20201104_01_T1",
"LC08_L1TP_033033_20201031_20201106_01_T1", "LC08_L1TP_033033_20201116_20201210_01_T1", "LC08_L1TP_033033_20201116_20210315_01_T1",
"LC08_L1TP_033033_20201202_20201217_01_T1", "LC08_L1TP_033033_20201202_20210312_01_T1", "LC08_L1TP_033033_20210103_20210309_01_T1",
"LC08_L1TP_033033_20210119_20210306_01_T1", "LC08_L1TP_033033_20210204_20210306_01_T1", "LC08_L1TP_033033_20210220_20210304_01_T1",
"LC08_L1TP_033033_20210308_20210317_01_T1", "LC08_L1TP_033033_20210324_20210401_01_T1", "LC08_L1TP_033033_20210409_20210416_01_T1",
"LC08_L1TP_033033_20210425_20210501_01_T1", "LC08_L1TP_033033_20210527_20210607_01_T1", "LC08_L1TP_033033_20210612_20210622_01_T1",
"LC08_L1TP_033033_20210628_20210707_01_T1", "LC08_L1TP_033033_20210714_20210721_01_T1", "LC08_L1TP_033033_20210730_20210804_01_T1",
"LC08_L1TP_033033_20210815_20210826_01_T1", "LC08_L1TP_033033_20210831_20210909_01_T1", "LC08_L1TP_033033_20210916_20210924_01_T1",
"LC08_L1TP_033033_20211002_20211013_01_T1", "LC08_L1TP_033033_20211018_20211026_01_T1", "LC08_L1TP_033033_20211103_20211109_01_T1",
"LC08_L1TP_033033_20211119_20211125_01_T1", "LC08_L1TP_033033_20211205_20211214_01_T1",
]

urls = []
for i in images:
    urls.append(["https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_B5.TIF".format(i, i),  # nir
                 "https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_B4.TIF".format(i, i),  # red
                 "https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_MTL.txt".format(i, i)])  # mtl

## Step 3: Create XArray datasets

In [ ]:
import xarray as xr

red = []
nir = []
for i in urls:
    red.append(rioxarray.open_rasterio(i[1], chunks={'band': 1, 'x': 1024, 'y': 1024}))
    nir.append(rioxarray.open_rasterio(i[0], chunks={'band': 1, 'x': 1024, 'y': 1024}))

## Step 4: Create a remote Dask cluster on Kubernetes via GKE

In [ ]:
# Create a Dask cluster on Kubernetes

from dask_kubernetes import KubeCluster, make_pod_spec
cluster = KubeCluster('worker-spec.yml')

In [ ]:
cluster.scale(15)  # Scale to a specific number of workers
# cluster.adapt(minimum=1, maximum=10)  # or dynamically scale based on current workload

In [ ]:
# Connect local Dask client to the remote cluster

from dask.distributed import Client
client = Client(cluster)

## Step 5: Rescale bands using Landsat metadata and Dask

The Landsat Level 1 images are delivered in a quantized format. This has to be [converted to top-of-atmosphere reflectance](https://landsat.usgs.gov/using-usgs-landsat-8-product) using the provided metadata. First we define convenience functions to load the rescaling factors and transform a dataset. The red band is band 4 and near infrared is band 5.

In [ ]:
def load_scale_factors(filename, band_number):
    metadata = {}
    response = requests.get(filename)
    data = response.text.splitlines()
    for line in data:
        name, var = line.partition("=")[::2]
        metadata[name.strip()] = var
    
    M_p = float(metadata["REFLECTANCE_MULT_BAND_{}".format(band_number)])
    A_p = float(metadata["REFLECTANCE_ADD_BAND_{}".format(band_number)])
    
    return M_p, A_p

def calculate_reflectance(ds, band_number, metafile):
    M_p, A_p = load_scale_factors(metafile, band_number)
    toa = M_p * ds + A_p
    return toa

red_toa = []
nir_toa = []
for i, j, k in zip(red, nir, urls):
    red_toa.append(calculate_reflectance(i, band_number=4, metafile=k[2]))
    nir_toa.append(calculate_reflectance(j, band_number=5, metafile=k[2]))

Because the transformation is composed of arithmetic operations, execution is delayed and the operations are parallelized automatically.

In [ ]:
print(red_toa[0].variable.data)

The resulting image has floating point data with magnitudes appropriate to reflectance. This can be checked by computing the range of values in an image using Dask:

In [ ]:
red_max, red_min, red_mean = dask.compute(
    red_toa[0].max(dim=['x', 'y']), 
    red_toa[0].min(dim=['x', 'y']),
    red_toa[0].mean(dim=['x', 'y'])
)
print(red_max.item())
print(red_min.item())
print(red_mean.item())

## Step 6: Calculate normalized difference vegetation index (NDVI) using Dask

Now that we have the image as reflectance values, we are ready to compute the NDVI using Dask.

$$
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}
$$

This highlights areas of healthy vegetation with high NDVI values, which appear as green in the image below.

In [ ]:
def compute_ndvi(nir_toa_single, red_toa_single, images):
    ndvi = (nir_toa_single - red_toa_single) / (nir_toa_single + red_toa_single)
    ndvi2d = ndvi.squeeze()

    fig = plt.figure(figsize=[12,12])
    im = ndvi2d.plot.imshow(cmap='BrBG', vmin=-0.5, vmax=1)
    plt.axis('equal')
    fig.savefig("output/" + images + ".png")
    plt.close(fig)
    return 1

In [ ]:
results = []
for i, j, k in zip(nir_toa, red_toa, images):
    y = dask.delayed(compute_ndvi)(i, j, k)
    results.append(y)

In [ ]:
from dask.distributed import progress

res = client.compute(results)
progress(res)

In [ ]:
# Close Dask client
client.close()

In [ ]:
# Close and terminate Dask cluster
cluster.close()